<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#imports" data-toc-modified-id="imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>imports</a></span></li><li><span><a href="#LI-with-barrier" data-toc-modified-id="LI-with-barrier-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>LI with barrier</a></span></li><li><span><a href="#general" data-toc-modified-id="general-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>general</a></span></li><li><span><a href="#DROP(HIGH)-->-DROP(LOW)" data-toc-modified-id="DROP(HIGH)-->-DROP(LOW)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>DROP(HIGH) -&gt; DROP(LOW)</a></span></li><li><span><a href="#SWA(5)-->-DROP(HIGH)" data-toc-modified-id="SWA(5)-->-DROP(HIGH)-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>SWA(5) -&gt; DROP(HIGH)</a></span></li><li><span><a href="#SWA(5)-->-DROP(LOW)" data-toc-modified-id="SWA(5)-->-DROP(LOW)-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>SWA(5) -&gt; DROP(LOW)</a></span></li><li><span><a href="#dump-to-the-disk" data-toc-modified-id="dump-to-the-disk-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>dump to the disk</a></span></li></ul></div>

# imports

In [1]:
import os
import pickle

import numpy as np
from tqdm import tqdm

from matplotlib import pyplot as plt

import torch
import numpy as np
from glob import glob

from matplotlib.pyplot import cm

In [31]:
def check_key_name(n):
#     return 0/0
    return 'linear' not in n and 'bn' not in n and 'shortcut.0' not in n
            
def make_flatten_vec(state_dict, layer=None):
    values = []
    if layer is None:
        for key, value in state_dict.items():
            if check_key_name(key):
                values.append(torch.flatten(value))
    else:
        values.append(torch.flatten(state_dict[layer]))
#             print('adding ', value.shape)
    vec = torch.cat(values, 0).to(torch.float64)
    return vec

In [8]:
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

def get_init_angle_dist(point_a:str, point_b:str):
    sd1 = torch.load(point_a)['state_dict']
    sd2 = torch.load(point_b)['state_dict']
    
#     print(sd1.keys()) 
#     print('-'*10)
#     print(sd2.keys())
    
    vec1 = make_flatten_vec(sd1).detach().cpu()
    vec2 = make_flatten_vec(sd2).detach().cpu()
    cdist = angle_between(vec1, vec2)
    return cdist

# LI with barrier

In [9]:
def read_angle_dist_from_track(checkpoint_dir: str, n_steps: int=20):
    base = os.path.join(checkpoint_dir, 'interp_result_{:5.4f}-{}.pt')
    pt_path0 = base.format(0.0, int(0.0))
    pt_path1 = base.format(1.0, int(1.0))

    angle = get_init_angle_dist(point_a=pt_path0, point_b=pt_path1)   
       
    return angle

In [10]:
def read_li_track(checkpoint_dir: str, n_steps: int=20):
    track = []
    for alpha in np.linspace(0.0, 1.0, n_steps + 1):
        base = os.path.join(checkpoint_dir, 'interp_result_{:5.4f}-{}.pt')
        pt_path = base.format(alpha, int(alpha))
        data = torch.load(pt_path)

        data['pnorm'] = np.linalg.norm(make_flatten_vec(data['state_dict']).cpu())

        del data['state_dict']

        data['alpha'] = alpha

        track.append(data)
    return track

In [11]:
def track_to_barrier(track_values: list, barrier_is_higher: bool=True):
    track_values = np.array(track_values)
    A = track_values[0]
    B = track_values[-1]
    
    alpha = np.linspace(0.0, 1.0, len(track_values))
    li = (1.0 - alpha) * A + alpha * B 
    if barrier_is_higher:
        return (track_values - li).clip(min=0.0).max()
    return (li - track_values).clip(min=0.0).max()

# general

In [20]:
USUAL_ELRS = [
    1e-4, 2e-4, 5e-4,
    1e-3, 2e-3, 5e-3,
    1e-2, 2e-2, 5e-2,
    1e-1, 2e-1, 5e-1,
    1
]

EDLRS = [1e-4, 2e-3]

# DROP(HIGH) -> DROP(LOW)

In [23]:
spt = 3
for i, elr in enumerate(USUAL_ELRS):
    gp = (i // spt)
    
    print("""python ./linear_interpolation_resnet18_cifar10_clean.py \\
    --gpu {} --elr {} --n_interp 20 --recalc_bn 1 \\
    --point_a ./finetune_high_dlr_2e-03/lr{}_checkpoint.pt \\
    --point_b ./finetune_low_dlr_1e-04/lr{}_checkpoint.pt \\
    --save ./Experiments/CONNECTIVITY_RN18C10_lri_{}_from_drop_2e-03_to_drop_1e-04/ && \\""".\
         format(gp, elr, 
                elr, 
                elr, 
                elr))
    print("""python ./calc_grad_norms_resnet18siaf_cifar10_clean.py \\
    --gpu {} \\
    --directory_with_checkpoints ./Experiments/CONNECTIVITY_RN18C10_lri_{}_from_drop_2e-03_to_drop_1e-04/ \\
    --train_mode 1 && \\""".\
         format(gp, 
                elr))
    if i % spt == spt - 1:
        print('\n\n')

python ./linear_interpolation_resnet18_cifar10_clean.py \
    --gpu 0 --elr 0.0001 --n_interp 20 --recalc_bn 1 \
    --point_a ./finetune_high_dlr_2e-03/lr0.0001_checkpoint.pt \
    --point_b ./finetune_low_dlr_1e-04/lr0.0001_checkpoint.pt \
    --save ./Experiments/CONNECTIVITY_RN18C10_lri_0.0001_from_drop_2e-03_to_drop_1e-04/ && \
python ./calc_grad_norms_resnet18siaf_cifar10_clean.py \
    --gpu 0 \
    --directory_with_checkpoints ./Experiments/CONNECTIVITY_RN18C10_lri_0.0001_from_drop_2e-03_to_drop_1e-04/ \
    --train_mode 1 && \
python ./linear_interpolation_resnet18_cifar10_clean.py \
    --gpu 0 --elr 0.0002 --n_interp 20 --recalc_bn 1 \
    --point_a ./finetune_high_dlr_2e-03/lr0.0002_checkpoint.pt \
    --point_b ./finetune_low_dlr_1e-04/lr0.0002_checkpoint.pt \
    --save ./Experiments/CONNECTIVITY_RN18C10_lri_0.0002_from_drop_2e-03_to_drop_1e-04/ && \
python ./calc_grad_norms_resnet18siaf_cifar10_clean.py \
    --gpu 0 \
    --directory_with_checkpoints ./Experiments/CONNE

In [32]:
interp_drophigh_droplow = dict()
for elr in tqdm(USUAL_ELRS): 
    interp_drophigh_droplow [elr] = dict()
    base = './Experiments/CONNECTIVITY_RN18C10_lri_{}_from_drop_2e-03_to_drop_1e-04'
    pt_path = base.format(elr)
    
    interp_drophigh_droplow[elr]['angle'] = read_angle_dist_from_track(pt_path)
    
    track = read_li_track(pt_path)
    interp_drophigh_droplow[elr]['track'] = track
    
    loss_barrier = track_to_barrier([x['loss_trainmode_train'] for x in track], barrier_is_higher=True)
    interp_drophigh_droplow[elr]['loss_barrier'] = loss_barrier
    
    lossts_barrier = track_to_barrier([x['test_res']['loss'] for x in track], barrier_is_higher=True)
    interp_drophigh_droplow[elr]['lossts_barrier'] = lossts_barrier
    
    testacc_barrier = track_to_barrier([x['test_res']['accuracy'] for x in track], barrier_is_higher=False)
    interp_drophigh_droplow[elr]['testacc_barrier'] = testacc_barrier
    
    loss_barrier = track_to_barrier([x['acc_trainmode_train'] for x in track], barrier_is_higher=False)
    interp_drophigh_droplow[elr]['trainacc_barrier'] = loss_barrier

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:50<00:00,  3.88s/it]


# SWA(5) -> DROP(HIGH)

In [28]:
spt = 3
for i, elr in enumerate(USUAL_ELRS):
    gp = (i // spt)
    
    print("""python ./linear_interpolation_resnet18_cifar10_clean.py \\
    --gpu {} --elr {} --n_interp 20 --recalc_bn 1 \\
    --point_a ./swa_from_200_for_5/lr{}_checkpoint.pt \\
    --point_b ./finetune_high_dlr_2e-03/lr{}_checkpoint.pt \\
    --save ./Experiments/CONNECTIVITY_RN18C10_lri_{}_from_swa_5_to_drop_2e-03/ && \\""".\
         format(gp, elr, 
                elr, 
                elr, 
                elr))
    print("""python ./calc_grad_norms_resnet18siaf_cifar10_clean.py \\
    --gpu {} \\
    --directory_with_checkpoints ./Experiments/CONNECTIVITY_RN18C10_lri_{}_from_swa_5_to_drop_2e-03/ \\
    --train_mode 1 && \\""".\
         format(gp, 
                elr))
    if i % spt == spt - 1:
        print('\n\n')

python ./linear_interpolation_resnet18_cifar10_clean.py \
    --gpu 0 --elr 0.0001 --n_interp 20 --recalc_bn 1 \
    --point_a ./swa_from_200_for_5/lr0.0001_checkpoint.pt \
    --point_b ./finetune_high_dlr_2e-03/lr0.0001_checkpoint.pt \
    --save ./Experiments/CONNECTIVITY_RN18C10_lri_0.0001_from_swa_5_to_drop_2e-03/ && \
python ./calc_grad_norms_resnet18siaf_cifar10_clean.py \
    --gpu 0 \
    --directory_with_checkpoints ./Experiments/CONNECTIVITY_RN18C10_lri_0.0001_from_swa_5_to_drop_2e-03/ \
    --train_mode 1 && \
python ./linear_interpolation_resnet18_cifar10_clean.py \
    --gpu 0 --elr 0.0002 --n_interp 20 --recalc_bn 1 \
    --point_a ./swa_from_200_for_5/lr0.0002_checkpoint.pt \
    --point_b ./finetune_high_dlr_2e-03/lr0.0002_checkpoint.pt \
    --save ./Experiments/CONNECTIVITY_RN18C10_lri_0.0002_from_swa_5_to_drop_2e-03/ && \
python ./calc_grad_norms_resnet18siaf_cifar10_clean.py \
    --gpu 0 \
    --directory_with_checkpoints ./Experiments/CONNECTIVITY_RN18C10_lri_0.0

In [33]:
interp_swa5_drophigh = dict()
for elr in tqdm(USUAL_ELRS): 
    interp_swa5_drophigh[elr] = dict()
    base = './Experiments/CONNECTIVITY_RN18C10_lri_{}_from_swa_5_to_drop_2e-03'
    pt_path = base.format(elr)
    
    interp_swa5_drophigh[elr]['angle'] = read_angle_dist_from_track(pt_path)
    
    track = read_li_track(pt_path)
    interp_swa5_drophigh[elr]['track'] = track
        
    loss_barrier = track_to_barrier([x['loss_trainmode_train'] for x in track], barrier_is_higher=True)
    interp_swa5_drophigh[elr]['loss_barrier'] = loss_barrier
    
    lossts_barrier = track_to_barrier([x['test_res']['loss'] for x in track], barrier_is_higher=True)
    interp_swa5_drophigh[elr]['lossts_barrier'] = lossts_barrier
    
    testacc_barrier = track_to_barrier([x['test_res']['accuracy'] for x in track], barrier_is_higher=False)
    interp_swa5_drophigh[elr]['testacc_barrier'] = testacc_barrier
    
    loss_barrier = track_to_barrier([x['acc_trainmode_train'] for x in track], barrier_is_higher=False)
    interp_swa5_drophigh[elr]['trainacc_barrier'] = loss_barrier

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:50<00:00,  3.86s/it]


# SWA(5) -> DROP(LOW)

In [30]:
spt = 3
for i, elr in enumerate(USUAL_ELRS):
    gp = (i // spt)
    
    print("""python ./linear_interpolation_resnet18_cifar10_clean.py \\
    --gpu {} --elr {} --n_interp 20 --recalc_bn 1 \\
    --point_a ./swa_from_200_for_5/lr{}_checkpoint.pt \\
    --point_b ./finetune_low_dlr_1e-04/lr{}_checkpoint.pt \\
    --save ./Experiments/CONNECTIVITY_RN18C10_lri_{}_from_swa_5_to_drop_1e-04/ && \\""".\
         format(gp, elr, 
                elr, 
                elr, 
                elr))
    print("""python ./calc_grad_norms_resnet18siaf_cifar10_clean.py \\
    --gpu {} \\
    --directory_with_checkpoints ./Experiments/CONNECTIVITY_RN18C10_lri_{}_from_swa_5_to_drop_1e-04/ \\
    --train_mode 1 && \\""".\
         format(gp, 
                elr))
    if i % spt == spt - 1:
        print('\n\n')

python ./linear_interpolation_resnet18_cifar10_clean.py \
    --gpu 0 --elr 0.0001 --n_interp 20 --recalc_bn 1 \
    --point_a ./swa_from_200_for_5/lr0.0001_checkpoint.pt \
    --point_b ./finetune_low_dlr_1e-04/lr0.0001_checkpoint.pt \
    --save ./Experiments/CONNECTIVITY_RN18C10_lri_0.0001_from_swa_5_to_drop_1e-04/ && \
python ./calc_grad_norms_resnet18siaf_cifar10_clean.py \
    --gpu 0 \
    --directory_with_checkpoints ./Experiments/CONNECTIVITY_RN18C10_lri_0.0001_from_swa_5_to_drop_1e-04/ \
    --train_mode 1 && \
python ./linear_interpolation_resnet18_cifar10_clean.py \
    --gpu 0 --elr 0.0002 --n_interp 20 --recalc_bn 1 \
    --point_a ./swa_from_200_for_5/lr0.0002_checkpoint.pt \
    --point_b ./finetune_low_dlr_1e-04/lr0.0002_checkpoint.pt \
    --save ./Experiments/CONNECTIVITY_RN18C10_lri_0.0002_from_swa_5_to_drop_1e-04/ && \
python ./calc_grad_norms_resnet18siaf_cifar10_clean.py \
    --gpu 0 \
    --directory_with_checkpoints ./Experiments/CONNECTIVITY_RN18C10_lri_0.000

In [34]:
interp_swa5_droplow = dict()
for elr in tqdm(USUAL_ELRS): 
    interp_swa5_droplow[elr] = dict()
    base = './Experiments/CONNECTIVITY_RN18C10_lri_{}_from_swa_5_to_drop_1e-04'
    pt_path = base.format(elr)
    
    interp_swa5_droplow[elr]['angle'] = read_angle_dist_from_track(pt_path)
    
    track = read_li_track(pt_path)
    interp_swa5_droplow[elr]['track'] = track
    
    loss_barrier = track_to_barrier([x['loss_trainmode_train'] for x in track], barrier_is_higher=True)
    interp_swa5_droplow[elr]['loss_barrier'] = loss_barrier
    
    lossts_barrier = track_to_barrier([x['test_res']['loss'] for x in track], barrier_is_higher=True)
    interp_swa5_droplow[elr]['lossts_barrier'] = lossts_barrier
    
    testacc_barrier = track_to_barrier([x['test_res']['accuracy'] for x in track], barrier_is_higher=False)
    interp_swa5_droplow[elr]['testacc_barrier'] = testacc_barrier
    
    loss_barrier = track_to_barrier([x['acc_trainmode_train'] for x in track], barrier_is_higher=False)
    interp_swa5_droplow[elr]['trainacc_barrier'] = loss_barrier

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:49<00:00,  3.83s/it]


# dump to the disk

In [35]:
barrier_setups = dict()
barrier_setups['interp_drophigh_droplow'] = interp_drophigh_droplow
barrier_setups['interp_swa5_drophigh'] = interp_swa5_drophigh
barrier_setups['interp_swa5_droplow'] = interp_swa5_droplow

In [36]:
with open('./resnet18_w64_cifar10_barrier_setups.pkl', 'wb') as f:
    pickle.dump(barrier_setups, f)